In [69]:
'''
beautifulsoup 把節點做成attribute方便定位，同時也提供遍歷/搜尋 節點的方法 (find, findall, select)。

而pyquery每次運行都回傳對應tag的節點。(如有複數個，可以用len()找總數，用hyperlinks.eq(n)看第N個節點)
註：pyquery屎一般的文檔：https://pythonhosted.org/pyquery/index.html#full-documentation

# pyquery 預設載入 urllib / request(如果有) ，所以可以直接傳入URL當作輸入
# pyquery 可以接受的輸入有 代碼 / URL / 文檔。
# 使用 doc = pq(['代碼' | url="https://google.com" | filename = "D:/demo.xml"]) 輸入檔案，並可以用parser關鍵字指定解析方式。
'''
import re,requests
from PIL import Image
from pyquery import PyQuery as pq

url = 'https://www.dcard.tw/f/plant'
urlBase = 'https://'
VisitedLinks = set()
i=0

## Create Permanent connection
rs = requests.Session()

## get the response
r = rs.get(url)
doc = pq(url=url)
print(type(doc))
title = doc('title')
hyperlinks= doc('a')
print(len(title),type(title), title.text())

for i in range(len(hyperlinks)):
    
    ## Check if the hypertext reference in tag "a" is valid.
    hlink = hyperlinks.eq(i).attr('href')
    if not hlink: continue
    valid = re.search(r'/p/\d+',hlink)
    if not valid: continue
        
    ## Check if the urlPost have been visited
    UrlPost = url+valid.group()
    if UrlPost in VisitedLinks: continue
    VisitedLinks.add(UrlPost)
    
    ## Get image in each post.
    doc2 = pq(url=UrlPost)
    pics = doc2('img')
    break
    for j in range(len(pics)):
        picurl = pics.eq(j).attr('src')
        if not picurl: continue 
        picurl = re.search('imgur.*',picurl)
        if not picurl: continue
        print(pics.eq(j).attr('src'))
        with rs.get(pics.eq(j).attr('src'),stream=True) as rPic:
            im = Image.open(rPic.raw)
            im.save('{}.'.format(i)+im.format)
            i += 1
        
    break    

<class 'pyquery.pyquery.PyQuery'>
1 <class 'pyquery.pyquery.PyQuery'> 植物板 | Dcard


In [8]:
# Using requests and beautifulsoup to get pictures on dcard/plant

# 原本想往曬貓版去做，但不會處理webp，所以先用plant版做。(這個版的圖床是imgur，比較簡單。)
# 作業繳交之後研究一下怎麼處理WEBP檔。
#
import requests, re
from PIL import Image
from bs4 import BeautifulSoup as bs


## Set the variables
url = 'https://www.dcard.tw/f/plant'
urlBase = 'https://'
VisitedLinks = set()
i=0

## Create Permanent connection
rs = requests.Session()

## get the response
r = rs.get(url)

## Parsing the response
soup = bs(r.text)
hyperlinks = soup.find_all('a')

print(type(soup.title), soup.title.text)

for hlink in hyperlinks:
    ## Check if the hypertext reference in tag "a" is valid.
    try:
        valid = re.search('/p/\d+',hlink['href'])
    except:
        continue
    if not valid: continue
    UrlPost = url+valid.group()
    
    ## Check if the urlPost have been visited
    if UrlPost in VisitedLinks: continue
    VisitedLinks.add(UrlPost)
    
    ## Get image in each post.
    rPost = rs.get(UrlPost)
    soup2 = bs(rPost.text)
    pics = soup2.find_all('img')
    for pic in pics:
        picurl = re.search(r'imgur.*',pic['src'])
        if not picurl: continue
        print(urlBase+picurl.group())
        with rs.get(urlBase+picurl.group(),stream=True) as rPic:
            im = Image.open(rPic.raw)
            im.save('{}.'.format(i)+im.format)
            i += 1
    break 


<class 'bs4.element.Tag'> 植物板 | Dcard


In [148]:
# pyquery 就是 python 版本的 jquery。語法幾乎是一樣的。所以可以查jquery大概怎麼用選擇器來學習。
# jquery 資料 https://www.w3schools.com/jquery/jquery_ref_selectors.asp

doc = pq(url=url)
##print(doc('a'))
#print(type(doc('a').eq(0).text()),doc('a').text()) #提供所有a標籤內含的 text (string)
#print(type(doc('a').html()),doc('a').html()) #提供第一個a標籤裡面的 html內容 (string)
#print(type(doc('[href]'))) # 找出所有包含 attribute "href"的標籤。

## traversal (找出每一個 a標籤，再一個一個對 href 屬性)
#for item in doc('a').items():
#    print(item.attr('href'))

# 直接查找 有href屬性，並且內容符合要求之格式的。
# (找屬性時用 '[屬性]'；找ID時用 '#ID的值'；找標籤時用 '標籤')
# ="格式" 表示跟後者完全一樣； !='格式' 表示不為後者；$='格式' 表示結尾為後者；^='格式' 表示開頭為後者；*='格式' 表示內容有後者。
# 回傳的值是一個 pyquery物件，所以還是要用 eq(i)配合 attr('href')去抓出各個 reference。 但可以省去正則表達式配對的麻煩。
d2=doc("[href*='/p/']")
print(d2)


<a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234385599"><span>有植物專家能幫幫我嗎~</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234391684"><span>#問植物專家 請問這是什麼植物？</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234385793"><span>請問大家這是什麼樹？#盆景#小樹</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234382363"><span>（問）#荷花</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234389928"><span>請問這是什麼植物呢？</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234386766"><span>#請問 #多肉 #大戟科 #銀角珊瑚</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234385322"><span>#蕨類 請益這是什麼蕨？？</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234373954"><span>救救我的黃金葛</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234370565"><span>#問（已解決）這是什麼樹/花</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234368559"><span>水培的龜背芋終於長根了！！</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234367462"><span>請問這兩盆仙人掌名字是什麼</span></a><a class="tgn9uw-3 bbdvDs" href="/f/plant/p/234365093"><span>#空氣鳳梨</spa